In [1]:
using Revise
using MeshCat
using RigidBodyDynamics
using RigidBodyDynamics: Bounds
using MeshCatMechanisms
using BilevelTrajOpt
using GeometryTypes

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/9dd84cf97c4fc16956f39829c81f3b6907c8fc83-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/9dd84cf97c4fc16956f39829c81f3b6907c8fc83-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

INFO: Loading HttpServer methods...


In [2]:
urdf = joinpath("..", "urdf", "panda", "panda_arm_box.urdf")
mechanism = parse_urdf(Float64, urdf)
box = findbody(mechanism, "box")
basejoint = joint_to_parent(box, mechanism)
floatingjoint = Joint(basejoint.name, frame_before(basejoint), frame_after(basejoint), QuaternionFloating{Float64}())
replace_joint!(mechanism, basejoint, floatingjoint)
position_bounds(floatingjoint) .= Bounds(-100, 100)
velocity_bounds(floatingjoint) .= Bounds(-100, 100);

6-element Array{RigidBodyDynamics.Bounds{Float64},1}:
 (-100.0, 100.0)
 (-100.0, 100.0)
 (-100.0, 100.0)
 (-100.0, 100.0)
 (-100.0, 100.0)
 (-100.0, 100.0)

In [3]:
function make_end_effector_controller(state::MechanismState,
                                      end_effector::RigidBody,
                                      end_effector_point::Point3D,
                                      desired_point::Point3D)
    
    mechanism = state.mechanism
    v̇ = similar(velocity(state))
    
    joint_path = path(mechanism, root_body(mechanism), end_effector) # from root to end effector
    Jp = point_jacobian(state, joint_path, transform(state, end_effector_point, root_frame(mechanism)))
        
    function controller!(τ, t, state)
        ef_point_in_world = transform_to_root(state, end_effector) * end_effector_point
        point_jacobian!(Jp, state, joint_path, ef_point_in_world)
        des_point_in_world = transform(state, desired_point, root_frame(mechanism))
        Kp = 100.
        Kd = 10.
        Δp = des_point_in_world - ef_point_in_world
        v̇ .= Kp * Array(Jp)' * Δp.v .- Kd .* velocity(state)
        τ .= inverse_dynamics(state, v̇)
        τ[1:7] .= 0. # the box
    end
end

state = MechanismState(mechanism)
desired_point_v = [1., 0., .25]
end_effector = findbody(mechanism, "panda_link8")
end_effector_point = Point3D(default_frame(end_effector), [0.,0.,0.])
desired_point = Point3D(root_frame(mechanism), desired_point_v)
point_ctrl! = make_end_effector_controller(state, end_effector, end_effector_point, desired_point);

In [ ]:
# q0 = [1.0,0.0,0.0,0.0,1.0,0.0,0.25,0.000915268,0.940503,0.020397,-1.54156,0.0731173,-2.20121,0.138407]
# v0 = vcat([0.,0.,0.,0.,0.,0.],zeros(7))
# set_configuration!(state, q0)
# set_velocity!(state, zeros(num_velocities(mechanism)))
# final_time = 5.0
# t, q, v = RigidBodyDynamics.simulate(state, final_time, point_ctrl!);

In [4]:
q0 = [1.0,0.0,0.0,0.0,1.0,0.0,0.25,0.000915268,0.940503,0.020397,-1.54156,0.0731173,-2.20121,0.138407]
v0 = vcat([0.,0.,0.,0.,0.,0.],zeros(7))
x0 = MechanismState(mechanism)
set_configuration!(x0, q0)
set_velocity!(x0, v0)
Δt = 0.01
N = 10 #250

# obstacles
μ = 0.01
motion_type = :xyz

point = Point3D(default_frame(findbody(mechanism, "floor")), [0.,0.,0.])
normal = FreeVector3D(default_frame(findbody(mechanism, "floor")), [0.,0.,1.])
hs = HalfSpace(point, normal)
floor = Obstacle(hs, μ, motion_type)

point = Point3D(default_frame(findbody(mechanism, "box")), [-.25,0.,0.])
normal = FreeVector3D(default_frame(findbody(mechanism, "box")), [-1.,0.,0.])
hs = HalfSpace(point, normal)
box = Obstacle(hs, μ, motion_type)

obstacles = [floor,box]
env = parse_contacts(mechanism, urdf, obstacles);

In [5]:
traj = BilevelTrajOpt.simulate(x0,env,Δt,N,point_ctrl!,implicit_contact=true);
q = [traj[1:num_positions(x0),i] for i in 1:N]
t = cumsum([Δt for i in 1:N]);


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************



LoadError: [91mInterruptException:[39m

In [ ]:
vis = Visualizer()
open(vis)
mvis = MechanismVisualizer(mechanism, URDFVisuals(urdf), vis);

In [ ]:
setanimation!(mvis, t, q)

In [ ]:
widget = manipulate!(mvis)